# Thème Image - TP3 - Introduction au filtrage

## Notebook
* A tout moment, vous pouvez savoir dans quel répertoire vous vous trouvez en utilisant la commande `pwd`et lister les fichiers qui s'y trouvent avec la commande `ls`.

* Accèdez au dossier de travail, pour cela utilisez les commandes `pwd` (print working directory), `cd` (change directory) et `ls`(list contents) pour naviguer sur le disque.

* Changez de repertoire jusqu'a voir tous les fichiers necessaires au TP.

* Familiarisez vous avec les fonctions mises à votre disposition dans le fichier `utils.py`; elles vous seront utiles pour la suite.

* Pour commencer, executez la cellule suivante afin d'avoir accès à votre espace disque.

* Vérifiez que vous êtes au bon endroit, et que tous les fichiers nécessaires au TP sont bien présents. Utilisez la commande `ls` pour cela.

In [1]:
pwd

'/home/luunpl/Study/L1/201_MAP/TP_IMAGE'

* Importez les modules dont vous aurez besoin en exécutant la cellule suivante (`Shift+Enter`).

In [2]:
import math
import statistics
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

plt.ion()

import sys
sys.path.append('..')
from utils import *

test_utils()

ModuleNotFoundError: No module named 'utils'

# Objectifs du TP
* Aborder concrètement la notion de filtrage, d'abord en 1D puis en 2D
* Etudier deux filtres de lissage : filtre moyenne et filtre gaussien
* Application : lissage d'une image bruitée
  Image originale & Image bruitée




* Comparer l'effet des deux filtres

# Exercice 1
## Filtrage 1D

On s'intéresse dans un premier temps au filtrage de signaux 1D.

Dans le cadre du filtrage numérique, on travaille sur des vecteurs de nombres.
L'idée du filtrage est de  promener une "fenêtre" sur chacun des éléments du vecteur, et obtenir une nouvelle valeur à partir des éléments de cette fenêtre.


In [3]:
# Creer un vecteur d'indices de taille N (N pourra etre modifie par la suite)
N = 10
s = np.arange(N)
print(f's= {s}')

s= [0 1 2 3 4 5 6 7 8 9]


## Construction de la fenêtre et parcours du signal

* Etant donné un indice $u$ et un entier $m$, comment construire le sous-tableau de $s$ contenant les $2m+1$ éléments suivants :
$s(u-m), ..., s(u), ..., s(u+m)$ ?

* Pour quelles valeurs de $u$ est-il possible de construire ce sous-tableau ?

In [ ]:
# Ecrire une fonction permettant d'extraire la fenetre de valeurs [u-m, ..., u, ..., u+m]
def get_window(vec, u, m):
    window = []
    # A COMPLETER
    for i in range(2*m+1):
        window.append(vec[u+i])
    return window

In [ ]:
# Tester la fonction: que se passe-t-il lorsque u=0 ou u=N-1?
# Autre cas problematiques?
test_win = get_window(s, 2, 2)
print(f'test_win= {test_win}')

In [ ]:
# Ecrire une fonction qui verifie les bornes de la fenetre
def check_and_get_window(vec, u, m):
    # Check
    # A COMPLETER
    if u - m < 0 or u + m >= len(vec):
        return False, None
    
    # Compute
    window = []
    # A COMPLETER
    for i in range(-m, m + 1):
        window.append(vec[u + i])
    return True, window

In [ ]:
# Tester check_and_get_window
print(check_and_get_window(s, 2,4))
s2 = np.arange(50)
print(check_and_get_window(s2, 30,10))

In [ ]:
# Ecrire une fonction qui extrait toutes les fenetres admissibles a partir du vecteur vec
def print_all_windows(vec, m):
    # A COMPLETER
    for u in range(len(vec)):
        ok, win = check_and_get_window(vec, u, m)
        if ok:
            print(f'u= {u} --> {win}')


* Pour un vecteur de taille 10, et un rayon $m = 2$ vous devriez obtenir les fenêtres suivantes :
- u= 2 --> [0, 1, 2, 3, 4]
- u= 3 --> [1, 2, 3, 4, 5]
- u= 4 --> [2, 3, 4, 5, 6]
- u= 5 --> [3, 4, 5, 6, 7]
- u= 6 --> [4, 5, 6, 7, 8]
- u= 7 --> [5, 6, 7, 8, 9]

In [ ]:
# Tester
print_all_windows(s, 2)

In [ ]:
# Refaire le test avec un tableau de valeurs 'physiques'
s2 = np.sin(s)  # np.sin transforme le tableau en [sin(0), sin(1), sin(2), ...]
print(f's2= {s2}')
print_all_windows(s2, 2)

# Exercice 2
## Filtre moyenne

L'application du filtre moyenne consiste à remplacer la valeur d'un élément par la moyenne arithmétique des valeurs sur une fenêtre centrée autour de cet élément.

* En vous inspirant de l'exercice précédent, écrives une fonction qui à partir d'un tableau $s$ renvoie le tableau $s_{mean}$ défini de la façon suivante :
  * si on peut définir une fenêtre de taille $2m+1$ autour de l'indice $u$, $s_{mean}[u]$ est la moyenne des éléments de la fenêtre,
  * sinon $s_{mean}[0] = 0$.

_**Indication** : Vous pourrez utiliser la fonction `check_and_get_window` pour extraire de manière sécurisée les fenêtres, et la fonction `statistics.mean()` pour calculer la moyenne._

_Exemple :_
L'application du filtre moyen dayon $m=1$ sur le vecteur $s = [0, 0, 0, 0, 0, 3, 3, 3, 3, 3]$ doit produire $s_{mean}= [0, 0, 0, 0, 1, 2, 3, 3, 3, 0]$.

* Que pensez-vous de ce résultat ?

In [ ]:
# Ecrire une fonction permettant d'appliquer un filtre moyen sur un vecteur
def mean_filter(vec, m):
    mean_vec = []  # Nouveau vecteur de valeurs
    # A COMPLETER
    for i in range(len(vec)):
        ok, win = check_and_get_window(vec,i,m)
        if ok:
            mean_val = statistics.mean(win)
        else:
            mean_val = 0.
        mean_vec.append(mean_val)
    return mean_vec


In [ ]:
# Definir un vecteur de valeurs
s = [0, 0, 0, 0, 0, 3, 3, 3, 3, 3]
print(f's= {s}')

# Lisser
s_mean = mean_filter(s, 1)

# Comparer
print(f's_mean= {s_mean}')
show_vectors(s, s_mean, labels=['s', 's_mean'], title='Vecteur test et moyenne')


* Essayer avec un vecteur plus long, contenant des valeurs 'physiques' et du bruit.

In [ ]:
# Generer le signal pur
M = 100
s = np.sin(2 * np.pi * np.arange(M) / M)

# Generer un bruit blanc (distribution aleatoire uniforme)
noise_magnitude = 0.3
noise = noise_magnitude * np.random.randint(-100, +100, M) / 100

# Appliquer le bruit au signal
s = s + noise

# Filtrer
s_mean = mean_filter(s, 5)

# Comparer
show_vectors(s, s_mean, title='Vecteur bruité et moyenne')

# Exercice 3
## Moyenne pondérée

La moyenne arithmétique accorde un poids égal à chacun des pixels sur une fenêtre donnée. On considère d'autres types de moyennes, dites pondérées,
où un poids différent peut être accordé à chaque pixel.

Dans un cadre totalement différent, pensez à la note du baccalauréat qui est obtenue en affectant des coeficients différents selon les matières, en faisant la somme de toutes ces notes, puis en divisant par la somme des coefficients
pour ramener la note sur 20.

Ici, on fait de même : étant donné un tableau $W$ de taille $2m+1$ et un vecteur $s$, on veut construire un vecteur $s2$ tel que pour chaque indice $u$, $s2[u]$ contient la moyenne pondérée par les poids $W$ sur la fenêtre
de taille $2m+1$ autour de l'élément d'indice $u$ si on peut définir cette fenêtre, et sinon $0$.

* Etant donné un vecteur $W$ de taille $2m+1$ qu'on suppose normalisé (dont la somme des coefficients vaut 1), et un tableau $s$ contenant un signal, comment calculer $s2$ la moyenne pondérée de $s$ par les poids $W$ ?

_**Indications** : la fonction numpy.sum() calcule la somme des éléments d'un tableau. L'opérateur * permet de multiplier terme à terme deux tableaux dans numpy. Voir l'exemple dans la cellule ci-dessous._

In [ ]:
# Exemple d'utilisation des opérateurs de numpy
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(a*b)
print(np.sum(a * b))

In [ ]:
# Ecrire une fonction qui calcule une moyenne ponderee en appliquant un filtre
def apply_filter(s, W):
    s2 = []
    # A COMPLETER
    m = int((len(W) - 1) / 2)  # Que se passe-t-il si le filtre a une taille paire ?
    for i in range(len(s)):
        ok, win = check_and_get_window(s, i, m)
        if ok:
            mean_val = np.sum(win * W)
        else:
            mean_val = 0.
        s2.append(mean_val)
    return s2

In [ ]:
# Definir un vecteur de valeurs
s = [0, 0, 0, 0, 0, 3, 3, 3, 3, 3]
print(f's= {s}')

# Definir un filtre de moyenne ponderee
W = np.array([0.5, 1.0, 0.5]) / 2.0
print(f'W= {W}')

# Tester
s_wmean = apply_filter(s, W)
show_vectors(s, s_wmean, title='Vecteur test et moyenne ponderee')

* Testez avec les poids $1/3*[1, 1, 1]$, vous devez retrouver le même résultat qu'à l'exercice précédent.

In [ ]:
# En utilisant le filtre moyen de l'exercice precedent, on doit retrouver le meme resultat
# A COMPLETER
W = np.ones(3) / 3
print(f'W= {W}')

s = [0, 0, 0, 0, 0, 3, 3, 3, 3, 3]
print(f's= {s}')

s2 = apply_filter(s, W)
print(f's_mean= {s2}')
show_vectors(s, s2, title='Vecteur test et moyenne ponderee')

* Que se passe-t-il si on ne normalise pas le filtre ?

In [ ]:
W = np.array([1.0, 1.0, 1.0])  # Pas de normalisation: / 3.0
s_wmean = apply_filter(s, W)
show_vectors(s, s_wmean, title='Vecteur test et filtre NON normalise')

# Exercice 4
## Filtre gaussien

La fonction gaussienne $f(x) = e^{-\frac{x^2}{2\sigma^2}}$ a une forme de chapeau qui est intéressante pour construire une moyenne pondérée avec une
valeur plus importante sur l'élément central de la fenêtre et une décroissance rapide et régulière sur les autres éléments.

De plus la largeur du chapeau est contrôlée par le paramètre $\sigma$ (sigma), ce qui permet d'obtenir facilement une famille de filtres de taille plus ou moins grande.

In [ ]:
# Fonction gaussienne
def f(x, sig):
    # A COMPLETER
    return np.exp(-0.5 * x**2 / sig ** 2)

In [ ]:
# Courbes gaussiennes pour quelques valeurs de sigma
x = np.arange(-5.0, +5.0, 0.1)

fig = plt.figure()
ax = fig.add_subplot()
for sigma in [1.5, 1.0, 0.5]:
  y = [f(x, sigma) for x in x]
  ax.plot(x, y, label=f'Sigma={sigma}')

_ = ax.legend()

L'idée du filtre gaussien consiste à prendre les valeurs de $f(x)$ pour $x$ parcourant des ensembles de la forme $\lbrace -L, \ldots, 0, \ldots, L\rbrace$ où $L$ est choisi tel que $f(L+1)$ soit négligeable devant $f(0) = 1$.

Par exemple pour $\sigma = 0.5$ (voir courbe ci-dessus), on a
$f(0) = 1, f(1) \approx 0.135, f(2) \approx 0.0003$, donc $f(2)$ est négligeable devant $f(0)$.

Si on prend $L=1$ les poids correspondant à $\sigma = 0.5$ sont
$[0.135, 1, 0.135]$ avant normalisation.



In [ ]:
simga = 0.5
print(f'f(0)= {f(0, sigma)}')
print(f'f(1)= {f(1, sigma)}')
print(f'f(2)= {f(2, sigma)}')

# Et evidemment
print('\nLa distribution gaussienne est paire, donc')
print(f'f(-1)= f(1)= {f(-1, sigma)}')
print(f'f(-2)= f(2)= {f(-2, sigma)}')

De manière générale, on se fixe un seuil $\varepsilon$ et pour une valeur de $\sigma$ donnée, et on choisit la taille $L$ du filtre de telle manière que $f(L) > \varepsilon$ et $f(L + 1) < \varepsilon$.

* Montrez que  $f(L) > \varepsilon \Longleftrightarrow L < \sigma\sqrt{-2\log(\varepsilon)}$.

* Modifiez la cellule suivante pour définir une fonction qui calcule le tableau $[f(-L), ..., f(0), ..., f(L)]$ où $L$ est le plus grand entier vérifiant $f(L) > \varepsilon$ en fonction de paramètres $sigma$ et $epsilon$. Cette fonction doit ensuite renvoyer le tableau normalisé (dont la somme des éléments est égale à 1).

In [ ]:
# Ecrire une fonction qui genere un filtre gaussien de parametre sigma de taille max
def get_gauss_filter(sigma, epsilon):
    # Trouver le plus grand entier L tel que f(L) > epsilon
    # f(L) < eps <==> abs(L) < sig sqrt(-2 ln(eps)) i.e., L < sig sqrt(-2 ln(eps)), puisque L>0
    # A COMPLETER
    L = int(sigma * np.sqrt(- 2 * np.log(epsilon)))
    
    # # Debug
    # print(f'Check f(L)   >  {epsilon}: {f(L, sigma)}')
    # print(f'Check f(L+1) <= {epsilon}: {f(L+1, sigma)}')

    # Calculer les poids
    w = []
    for i in range(-L, +L + 1):
        # A COMPLETER
        w.append(f(i,sigma))
        
    # Normaliser les poids pour obtenir le filtre
    w_sum = sum(w)
    # A COMPLETER
    fil = np.array(w) / w_sum
    return fil

* On fixe $\varepsilon = 10^{-3}$.

* Vous pouvez tester votre fonction pour $\sigma = 0.5$ et $epsilon=0.001$ `get_gauss_filter` doit renvoyer $[0.106507 0.786986 0.106507]$. Remarquez qu'on a bien $\sum_i g_i = 1$.

_**IMPORTANT** : Vous devez avoir terminé cet exercice et obtenu le résultat demandé car vous aurez besoin de la fonction `get_gauss_filter` pour le reste de cette séance et (au moins) la séance suivante._

In [ ]:
# On se donne un epsilon
eps = 0.001

# Test
get_gauss_filter(0.5, 0.001)

* Remplissez le tableau suivant qui indique la taille du tableau en fonction de quelques valeurs de $\sigma$ :
  * $\sigma = 0.5 \Rightarrow size = ?$
  * $\sigma = 1.0 \Rightarrow size = ?$
  * $\sigma = 1.5 \Rightarrow size = ?$
  * $\sigma = 2.0 \Rightarrow size = ?$

Ici, $size$ est la taille du filtre, c'est-à-dire la taille du tableau, c'est-à-dire $2*L + 1$.

In [ ]:
# Calculer les longueurs des filtres gaussiens pour les valeurs de sigma suivantes
sigma = 0.5
size = int(sigma*np.sqrt(-2*np.log(eps)))
print(f'Longueur du filtre (2*L+1) : {size}')
sigma = 1
size = int(sigma*np.sqrt(-2*np.log(eps)))
print(f'Longueur du filtre (2*L+1) : {size}')
sigma = 1.5
size = int(sigma*np.sqrt(-2*np.log(eps)))
print(f'Longueur du filtre (2*L+1) : {size}')
sigma = 2.
size = int(sigma*np.sqrt(-2*np.log(eps)))
print(f'Longueur du filtre (2*L+1) : {size}')


# Exercice 5
## Lissage d'un signal bruité

Nous allons extraire un signal bruité à partir de l'image papillon_noise.bmp. La cellule ci-dessous extrait la ligne 128 de l'image et l'affiche sous forme de courbe.

In [ ]:
# Charger l'image bruitee
img = load_and_show_image('./papillon_noise.bmp', title='Image source bruitee')

# Extraire la ligne 128
row = img[128, :]

# Afficher le vecteur
show_vectors(row, title="Ligne extraite de l'image")


* Testez l'effet du filtre moyenne sur le vecteur $row$ en augmentant progressivement la taille de la fenêtre : 3, 5, 11, 21.

In [ ]:
# Lisser avec filtres moyens de taille 3, 5, 11, 21
mean_filtering = [row]
for size in [3, 5, 11, 21]:
    # A COMPLETER
    mean_fil = np.ones(size) / size
    mean_row = apply_filter(row, mean_fil)
    mean_filtering.append(mean_row)

    # Afficher chaque paire
    show_vectors(row, mean_row, title=f'Lissage moyen, size={size}')


* On peut afficher tous les signaux les uns sur les autre, mais c'est moins lisible...

In [ ]:
show_vectors(*mean_filtering)

* Testez l'effet du filtre Gaussien sur le vecteur $row$ en augmentant progressivement la valeur de $\sigma$ : 0.5, 1, 3, 5.

In [ ]:
# Lisser avec des filtres gaussiens, avec sigma= 0.5, 1, 3, 5
gauss_filtering = [row]
for sigma in [0.5, 1, 3, 5]:
    # A COMPLETER
    gauss_fil = get_gauss_filter(sigma, eps)
    gauss_row = apply_filter(row, gauss_fil)
    gauss_filtering.append(gauss_row)

    # Afficher chaque paire
    show_vectors(row, gauss_row, title=f'Lissage gaussien, sigma={sigma}, size={len(gauss_fil)}')


In [ ]:
# La encore, on peut afficher tout le monde sur la même figure... mais c'est un peu fouillis
show_vectors(*gauss_filtering)

# Exercice 6
## Filtrage 2D

On se place maintenant dans le cadre du filtrage appliqué aux images. Le signal ici est constitué des valeurs des pixels dans un tableau à deux dimensions et les fenêtres que l'on va déplacer sur l'image sont des carrés centrés sur le pixel considéré.

## Construction de la fenêtre et parcours de l'image

Soit $im$ un tableau de taille $M \times N$ contenant une image.

* Etant donné un pixel désigné par un couple d'indices $(u,v)$ et un entier $m$, comment construire le sous-tableau de $im$ contenant les $(2m+1)^2$ éléments autour du pixel ?

* Pour quelles valeurs de $(u,v)$ est-il possible de construire ce sous-tableau ?

In [ ]:
# Ecrire une fonction qui extrait une fenetre centree sur (u, v) taille (2*m + 1)x(2*m + 1)
def check_and_get_window_2D(img, u, v, m):
    # Check
    # A COMPLETER
    if u - m < 0 or u + m >= img.shape[0]:
        return False, None
    
    # Check
    # A COMPLETER
    if v - m < 0 or v + m >= img.shape[1]:
        return False, None
    
    # Compute
    window = np.empty((2*m+1, 2*m+1))
    for i in range(-m, +m+1):
        for j in range(-m, +m+1):
            # A COMPLETER
            window[m+i, m+j] = img[u+i,v+j]
    return True, window

* Testez votre fonction à l'aide de la cellule ci-dessous.

In [ ]:
# Construire un tableau 2D d'indices
im = np.zeros([10, 10])
for i in range(im.shape[0]):
    for j in range(im.shape[1]):
        im[i, j] = i*10 + j

print(im)

# Test 1
ok, win = check_and_get_window_2D(im, 1, 1, 2)
print(f'Should be False: {ok}')
print(win)

# Test 2
m = 2
ok, win = check_and_get_window_2D(im, 3, 3, m)
print(f'Should be True: {ok}')
print(f'Window (size= {2*m+1}x{2*m+1}) should be centered on element "33"=\n{win}')

# Autres tests
# A completer...

* Initialisez l'image $im$ en exécutant la cellule ci-dessous.

In [ ]:
# Test 3
im = np.array([[1, 0, 0, 0],
               [0, 2, 0, 0],
               [0, 0, 3, 0],
               [0, 0, 0, 4]])


* Trouvez toutes les fenêtres de taille 3x3 de ce tableau.

Vous devez obtenir le resultat suivant :

    -----------------
    [[1. 0. 0.]
     [0. 2. 0.]
     [0. 0. 3.]]
    -----------------
    [[0. 0. 0.]
     [2. 0. 0.]
     [0. 3. 0.]]
    -----------------
    [[0. 2. 0.]
     [0. 0. 3.]
     [0. 0. 0.]]
    -----------------
    [[2. 0. 0.]
     [0. 3. 0.]
     [0. 0. 4.]]

In [ ]:
for u in range(im.shape[0]):
    for v in range(im.shape[1]):
        # A COMPLETER
        ok, win = check_and_get_window_2D(im, u, v, 1)
        if ok:
            print('-----------------')
            print(win)

# Exercice 7
## Filtre moyenne

De même qu'en 1D, l'application de ce filtre consiste à calculer la nouvelle valeur d'un pixel comme étant la  moyenne des valeurs sur une fenêtre centrée autour de lui.

* En vous inspirant de l'exercice précédent, écrivez une fonction qui à partir d'une image $im$ renvoie l'image $im2$ définie de la façon suivante :
  * pour chaque couple d'indices $(u,v)$, $im2[u,v]$ contient la moyenne de la fenêtre de taille $(2m+1)^2$ autour de $im(u,v)$ si la fenêtre est définie, et sinon $0$.

* Vous pouvez tester avec une image simple contenant un carré noir sur fond blanc.

* Vous pouvez faire varier la taille de la fenêtre et observer le résultat.


In [ ]:
# Ecrire une fonction qui applique un filtre moyen a une image
def mean_filter_2D(img, m):
    # Creer une image vide de la meme taille que l'image en entree
    mean_img = np.empty(img.shape)

    # Appliquer le filtre
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # A COMPLETER
            ok, win = check_and_get_window_2D(img, i, j, m)
            if ok:
                mean_img[i,j] = win.sum()/((2*m+1)**2)
            else:
                mean_img[i,j] = 0.
    return mean_img

In [ ]:
# Creer une image d'un carre noir sur fond blanc
square = np.ones((10, 10)) * 255
square[3:7, 3:7] = 0

# Afficher
print(square)
show_image(square, title='Image carre test')

# Test 1
mean_square = mean_filter_2D(square, 1)
print(mean_square)
show_image(mean_square, title='Image carre test, filtre moyen 3x3')

* Faites de même avec l'image papillon.bmp, avec des filtres 3x3 et 11x11.

* Que pensez-vous du résultat pour $m=5$ ?

In [ ]:
# Charger l'image
test_img = load_and_show_image('./papillon.bmp', title='Image')

# Test 2
# A COMPLETER
mean_test_img = mean_filter_2D(test_img, 1)
show_image(mean_test_img, title='Image, filtre moyen 3x3')

# Test 3
# A COMPLETER
mean_test_img = mean_filter_2D(test_img, 5)
show_image(mean_test_img, title='Image, filtre moyen 11x11')


# Exercice 8
## Moyenne pondérée 2D

On étend en 2D la moyenne pondérée définie dans l'exercice 3.

* Etant donné une matrice $W$ de taille $(2m+1)^2$ qu'on suppose normalisée, et une image $im$ contenant un signal, comment calculer la moyenne pondérée par les poids $W$ ?

_**Indications** : l'opérateur numpy $*$ réalise le produit terme à terme de deux tableaux, et la fonction $np.sum$ renvoie la somme des éléments d'un tableau._

* Modifiez la fonction dans la cellule ci-dessous pour qu'elle réalise l'opération de moyenne pondérée par les poids $W$ sur les éléments de l'image $im$ et renvoie le résultat.

In [ ]:
# Ecrire une fonction qui applique un filtre en 2D a une image
def apply_filter_2D(im, W):
    fil_img = np.empty(im.shape)
    m = int((W.shape[0] - 1) / 2)
    # Que se passe-t-il si le filtre a une taille paire ?
    # Quels autres checks faudrait il mettre en place?

    # Calculer l'image filtree
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            # A COMPLETER
            ok, win = check_and_get_window_2D(im, i, j, m)
            if ok:
                mean_val = np.sum(win * W)
            else:
                mean_val = 0.
            fil_img[i, j] = mean_val
    return fil_img

* Testez avec les poids $W = (1/9)*np.ones(3,3)$, vous devez retrouver le même résultat qu'à l'exercice précédent.

In [ ]:
# Creer un filtre moyen 3x3 (m=1)
W = np.ones((3, 3)) / 9
print(f'W=\n{W}')

# Test 1: carre noir sur fond blanc
square = np.ones((10, 10)) * 255
square[3:7, 3:7] = 0
print(f'square=\n{square}')
show_image(square, title='Image carre test')
#
# A COMPLETER
mean_square = apply_filter_2D(square, W)
print(f'mean_square=\n{mean_square}')
show_image(mean_square, title='Image carre test + filtre pondere 3x3')

* Faites de même avec l'image papillon.bmp.

In [ ]:
# Test 2
test_img = load_and_show_image('./papillon.bmp', title='Image')
# A COMPLETER
W = np.ones([3,3])/9
mean_test_img = apply_filter_2D(test_img, W)
show_image(mean_test_img, title='Image + filtre pondere 3x3')

# Exercice 9
## Filtre gaussien

On peut étendre à deux dimensions la fonction gaussienne vue auparavant :
$f_{2D}(x,y) = e^{-\frac{x^2+y^2}{2\sigma^2}}$. Elle possède les mêmes bonnes propriétés pour obtenir des poids : valeur plus importante sur l'élément central de la fenêtre et décroissance régulière; de plus elle ne possède pas de direction privilégiée.

Par ailleurs, on peut observer que $f_{2D}(x,y) = e^{-\frac{x^2}{2\sigma^2}}e^{-\frac{y^2}{2\sigma^2}} = f(x)f(y)$, ce qui implique qu'on peut construire le filtre gaussien $G2D$ correspondant à $f_{2D}$ à partir du filtre gaussien 1D $G$ construit plus haut :
$\tt \forall (i,j), G2D(i,j) = G(i)*G(j)$

* Modifiez la cellule ci-dessous pour qu'elle renvoie le tableau $G2D$ décrit ci-dessus. Il y a (au moins) deux façons de faire : soit en réutilisant les termes du filtre 1D, soit en repartant du calcul de $L$ puis en échantillonnant la fonction $f_{2D}$.

* Pour $\sigma=0.5$ et $\epsilon=0.001$ e tableau doit contenir les valeurs suivantes :

|           |           |           |
|-----------|-----------|-----------|
| 0.0113437 | 0.0838195 | 0.0113437 |
| 0.0838195 | 0.619347  | 0.0838195 |
| 0.0113437 | 0.0838195 | 0.0113437 |


* Remarquez qu'on a bien $\sum_{i, j} \texttt{G2D}_{ij} = 1$.

In [ ]:
# Ecrire une fonction gaussienne 2D
def f_2D(x, y, sig):
    return f(x, sig)*f(y, sig)


# Ecrire une fonction qui calcule une filtre gaussien en 2D
def get_gauss_filter_2D(sigma, epsilon):
    # A COMPLETER
    # Trouver le plus grand entier L tel que f(L) > epsilon
    L = int(sigma*np.sqrt(-2*np.log(epsilon)))

    # # Debug
    # print(f'Check f(L)   >  {epsilon}: {f(L, sigma)}')
    # print(f'Check f(L+1) <= {epsilon}: {f(L+1, sigma)}')

    # Calculer les poids
    w = np.empty((2*L+1, 2*L+1))
    for i in range(-L, L+1):
        for j in range(-L, L+1):
            w[L+i,L+j] = f_2D(i,j,sigma)

    # Normaliser les poids pour obtenir le filtre
    fil = w/w.sum()
    return fil
    
# Test
G2D = get_gauss_filter_2D(sigma=0.5, epsilon=0.001)
print(f'G2D=\n{G2D}')
print(f'somme= {np.sum(G2D)}')

* Vous pouvez maintenant utiliser la fonction de l'exercice précédent pour appliquer le filtre gaussien sur l'image papillon.bmp.

* Vous pouvez faire varier $\sigma$.

* Que pensez-vous du résultat pour $\sigma = 2.5$ en comparaison avec le résultat avec un filtre moyenne de grande taille ?

* Comparer a l'image produite par un filtre moyen de grande taille (m=5) plus haut.

In [ ]:
# Test 2
im = load_and_show_image('./papillon.bmp', title='Image')
# A COMPLETER
G2D = get_gauss_filter_2D(sigma=0.5, epsilon=0.001)
im2 = apply_filter_2D(im, G2D)

print(f'G2D {G2D.shape}=\n{G2D}')
show_image(im2, title='Image + filtre gaussien 2D')

# Exercice 10
## Lissage d'une image bruitée

On reprend l'image bruitée papillon_noise.bmp.

* Appliquez le filtre moyenne à l'image bruitée pour plusieurs tailles de fenêtres.

* Faites de même avec le filtre gaussien pour plusieurs valeurs de $\sigma$.

* Avec quel filtre obtient-on le meilleur résultat ?

In [ ]:
# Charger l'image bruitee
img = load_and_show_image('./papillon_noise.bmp', title='Image bruitee')

# Lisser avec des filtres gaussiens, avec sigma= 0.5, 1, 3, 5
gauss_sizes = []
for sigma in [0.5, 1, 3, 5]:
    # Generer le filtre gaussien
    # A COMPLETER
    gauss_fil = get_gauss_filter_2D(sigma=sigma, epsilon=0.001)
    size = gauss_fil.shape[0]
    print(f'Filtre gaussien, sigma= {sigma}, {size} x {size}')

    # Garder en memoire ses dimensions
    gauss_sizes.append(size)

    # Appliquer a l'image
    # A COMPLETER
    gauss_img = apply_filter_2D(img, gauss_fil)
    
    # Afficher l'image
    show_image(gauss_img, title=f'Image bruitee + filtre gaussien {size}x{size} sigma={sigma}')

# Lisser avec des filtres moyens de MEME taille que les filtres gaussiens utilises plus haut
for size in gauss_sizes:
    # Generer le filtre moyen
    # A COMPLETER
    mean_fil = np.ones((size,size)) / (size**2)
    print(f'Filtre moyen {size} x {size}')

    # Appliquer a l'image
    # A COMPLETER
    mean_img = apply_filter_2D(img, mean_fil)
    
    # Afficher l'image
    show_image(mean_img, title=f'Image bruitee + filtre moyen {size}x{size}')

# Exercice 11
## Technique du masque flou

La technique du masque flou consiste à utiliser une version floue d'une image pour augmenter la netteté et faire ressortir ses détails :

* Soit $I$ une image et $I_f$ sa version floue ; le masque est $I_m = I - I_f$, il contient les éléments ``non flous'' de l'image, donc ses détails.

* L'image traitée est obtenue en combinant l'image initiale $I$ et le masque $I_m$. Nous nous limiterons aux combinaisons de la forme $I + \alpha I_m$ pour $\alpha>0$ mais il en existe d'autres.

**REMARQUE** : A partir de maintenant nous utiliserons la fonction `signal.convolve2d()` de la librairie `scipy` pour appliquer un filtre $H$ à une image $im$.

* Dans la cellule ci-dessous complétez la fonction qui prend en entrée une image $im$ et le paramètre $\sigma$ afin qu'elle renvoie le masque obtenu par différence entre $im$ et de sa version floue obtenue par convolution avec le filtre gaussien de paramètre $\sigma$.

In [ ]:
# Ecrire une fonction qui calcule le masque flou d'une image img := img - img * G2D(sigma)
def get_mask(im, sigma):
    # Calculer le filtre
    # A COMPLETER
    fil = get_gauss_filter_2D(sigma=sigma, epsilon=0.001)
    
    # Calculer la convolution
    # Differences avec les calculs precedents:
    # - bien plus rapide
    # - le masque est retourne afin de respecter la definition de l'operateur de convolution
    # - gestion des bords differente (on 'rajoute' des pixels nuls lorsque le masque deborde de l'image)
    # ==> Pourquoi ce retournement ? est-ce que cela change notre resultat ?
    print_timestamp('Start convolve 2D')
    gauss_im = signal.convolve2d(im, fil, mode='same')
    print_timestamp('End convolve 2D')

    # Soustraire: attention ici le masque n'est plus veritablement une image car il peut contenir
    # des valeurs negatives
    # A COMPLETER
    mask = im - gauss_im
    
    return mask

* Ecrivez une fonction qui applique le masque flou à une image.

In [ ]:
# Ecrire une fonction qui applique un masque flou avec un coefficient alpha
def apply_mask(img, mask, alpha):
    """
    img: input image
    mask: input mask
    alpha: coefficient
    returns: masked image
    """
    # Que faut-il verifier ici pour que le programme ne crashe pas?
    # A COMPLETER
    if img.shape != mask.shape:
        raise ValueError('Shape mismatch!')

    if False:  # Testez les deux versions en changeant ce flag à False
      # Version pythonesque: sans boucles
      # A COMPLETER
      new_img = int(img + alpha * mask)
      
    else:
      # Version 'white box' (sans doute moins performante), avec des boucles for
      new_img = np.empty(img.shape)
      for i in range(img.shape[0]):
          for j in range(img.shape[1]):
              # A COMPLETER
              new_val = img[i,j] + alpha * mask[i,j]
              new_img[i, j] = int(new_val)
              
    return new_img

Vous pouvez tester cette fonction sur une image contenant deux "gros pixels" (voir cellule ci-dessous).

1. Generer l'image test
2. Calculer le masque avec $\sigma=1.5$, et l'afficher
3. Appliquer le masque avec $\alpha=1.2$, et afficher le resultat

In [ ]:
# Test 1
im = np.ones((50, 100)) * 200
im[:, 50:100] = 100
show_image(im, title='Image test, 2 gros pixels')
#
# A COMPLETER
mask = get_mask(im, sigma=1.5)
show_image(mask, check_bounds=False, title='Masque flou de Image test (contient valeurs signees: + et -)')
#
# A COMPLETER
im_masked = apply_mask(im, mask, alpha=1.2)
show_image(im_masked, title='masque flou applique a Image test')

Pour comprendre le fonctionnement du filtre, nous allons visualiser ce qui se passe sur une ligne horizontale de l'image.

Le code dans la cellule ci-dessous prend la ligne du milieu (y=50) de l'image et du masque et visualise les effets de l'opération.

* Expliquer pourquoi cet operateur augmente le contraste entre les regions sombre et claire.

In [ ]:
# Extrait la ligne du milieu de l'image avant et apres masquage
im_row = im[25, :]
mask_row = mask[25, :]

# Retrouve l'image filtree par le filtre gaussien
# A COMPLETER
gauss_row = im_masked[25,:]

# Afficher les 3 lignes ensemble
show_vectors(im_row, gauss_row, mask_row,
             labels=['ligne source', 'ligne lissée', 'masque flou'],
             title='Ligne test, lissée et masque flou')

# Effet du masque flou sur l'image pour diffèrentes valeurs de alpha
test_alphas = []
labels_alpha = []
for alpha in [0.5, 1.2, 2.0, 3.0]:
    im_masked_row = np.clip(im_row + alpha * mask_row, 0, 255)
    test_alphas.append(im_masked_row)
    labels_alpha.append(f'Alpha={alpha}')

show_vectors(*test_alphas, labels=labels_alpha, title='Image corrigée pour diffèrentes valeurs de alpha')

* Testez avec l'image sweets.bmp pour différentes valeurs de $\alpha$ : 1, 5, 10.

In [ ]:
# Test 2
test_img = load_and_show_image('./sweets.bmp', title='Bonbons')
#
# A COMPLETER
mask = get_mask(test_img, sigma=1.5)

show_image(mask, check_bounds=False, title='Masque flou de Bonbons')
#
for alpha in [1, 5, 10]:
    # A COMPLETER
    test_img_masked = apply_mask(test_img, mask, alpha)
    test_img_masked = np.clip(test_img_masked, 0, 255)
    #test_img_masked = (test_img_masked - test_img_masked.min())/(test_img_masked.max() - test_img_masked.min())*255
    show_image(test_img_masked, title=f"Sweets avec alpha={alpha}.")
